#### Packages

In [15]:
import pandas as pd
import numpy as np
import liblinear as ll
from liblinear import *
import itertools
from scipy.sparse import lil_matrix

#### Read Data

In [16]:
df = pd.read_csv("ML_HW4_data_train.dat", sep = "\t", header = None)
df.insert(loc=0, column="x0", value=1)
df.columns =['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'sign']
X = df.iloc[:, :-1].to_numpy()
y = df.iloc[:, -1].to_numpy()
print(X.shape)

df_t = pd.read_csv("ML_HW4_data_test.dat", sep = "\t", header = None)
df_t.insert(loc=0, column="x0", value=1)
X_t = df_t.iloc[:, :-1].to_numpy()
y_t = df_t.iloc[:, -1].to_numpy()

(200, 11)


##### Polynomial Transformation

In [17]:
def polynomial_features(X, degree):
    n_samples, n_features = X.shape
    combinations = itertools.combinations_with_replacement(range(n_features), degree)
    new_n_features = sum(1 for _ in combinations)
    X_poly = lil_matrix((n_samples, new_n_features), dtype=np.float32)
    
    for i, power in enumerate(combinations):
        X_poly[:, i] = np.prod(X[:, power], axis=1)

    return X_poly

X_train = polynomial_features(X, 4)
X_test = polynomial_features(X_t, 4)

#### 12

In [18]:
lambdas = [-6, -3, 0, 3, 6]
for i in range(len(lambdas)):
    lambdas[i] = 10**lambdas[i]

    
# Evaluate the out-of-sample error for each value of lambda
E_out = []
for lambda_ in lambdas:
    param = f'-s 0 -c {1/(lambda_*len(X))} -q'
    model = ll.train(y_train, X_train, param)
    _, acc, _ = ll.predict(y_test, X_test, model, '-q')
    E_out.append(1-acc[0]/100)

# Select the best value of lambda
best_lambda = lambdas[np.argmin(E_out)]

AttributeError: module 'liblinear' has no attribute 'train'